Because signal.2dcorrelate is extremely slow. It would be faster using for-loop and np.correlate for a one-short-side 2d output. So do convolution.

In [1]:
import numpy as np
from numba import jit
import matplotlib.pyplot as plt
import time as toc

In [2]:
# @jit
def STcorrelate(x,y,tstep,dx,dt,c=1):
    if c ==1:
        x_c = x-np.mean(x)
        y_c = y-np.mean(y)
    else:
        x_c = x.copy()
        y_c = y.copy()
    STkernel = np.zeros([len(tstep),max(x.shape[1], y.shape[1])])
    time_axis = tstep*dt
    space_axis = np.arange(max(x.shape[1], y.shape[1]))*dx
    space_axis = space_axis-np.mean(space_axis)
    for i in range(len(tstep)):
        for ii in np.arange(max(-tstep[i],0), y_c.shape[0]-1+min(-tstep[i], 0)):
            STkernel[i,:] += np.correlate(y_c[ii,:],x_c[tstep[i]+ii, :],'same')*dx*dt
    return STkernel, (time_axis, space_axis)

In [3]:
# @jit
def STconvolve(x,k,dx,dt):
    output = np.zeros_like(x)
    for i in range(x.shape[0]):
        for ii in np.arange(max(i-k.shape[0]+1, 0), i+1):
            output[i,:] += np.convolve(x[ii,:],k[i-ii, :],'same')*dx*dt
    return output

In [4]:
# @jit
def ST_STA(x,y,tstep,dx,dt,c = 0):
    if c == 1:
        x_c = x-np.mean(x)
        y_c = y-np.mean(y)
    else:
        x_c = x.copy()
        y_c = y.copy()
    STkernel = np.zeros([len(tstep),max(x.shape[1], y.shape[1])])
    Denominator = np.zeros([len(tstep),max(x.shape[1], y.shape[1])])
    time_axis = tstep*dt
    space_axis = np.arange(max(x.shape[1], y.shape[1]))*dx
    space_axis = space_axis-np.mean(space_axis)
    for i in range(len(tstep)):
        for ii in np.arange(max(-tstep[i],0), y_c.shape[0]-1+min(-tstep[i], 0)):
            STkernel[i,:] += np.correlate(y_c[ii,:],x_c[tstep[i]+ii, :],'same')*dx*dt
            Denominator[i,:] += np.correlate(y_c[ii,:],np.ones_like(x_c[tstep[i]+ii, :]),'same')*dx*dt
    return STkernel/Denominator, (time_axis, space_axis)